# Visualizing HRRR Model Data Over a General Area Using Herbie
In this notebook we will download and plot HRRR data.

In [1]:
#Import Modules
from herbie import Herbie
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import imageio
import glob

### Downloading the Data
Here you can input the date and pressure level, and then choose the number of forecast hours.

In [2]:
yr = input('Year (yyyy):')
m = input('Month (mm):')
d = input('Day (dd):')
level = input('Enter pressure level (surface, 1000 mb, 925 mb, 850 mb, etc.):')
fxx = input('Choose the number of forecast hours (up to 48):')
date = (f'{yr}-{m}-{d}')
fh = []
H = []
ds = []
for a in range(0, int(fxx) + 1):
    #Create a list of the forecast hours in "HH" format, this allows
    #for saving all images in order later for creating the gif
    if a <10:
        fh.append(f'0{a}')
    else:
        fh.append(f'{a}')
    if level == str('surface'):
        product = "sfc"
    else:
        product = "prs"
    #Read in the data
    H.append(Herbie(f"{yr}-{m}-{d}",  # model run date
        model="hrrr",  # model name
        product=product, #product
        fxx=a,  # forecast lead time
              ))
    if level == str('surface'):
        ds.append(H[a].xarray(f":{level}:", remove_grib=False)[0])
    else:
        ds.append(H[a].xarray(f":{level}:", remove_grib=False))
        #Calculate wind speed and add it to the dataset
        ds[a]['wspd'] = (('y', 'x'), (mpcalc.wind_speed(ds[a].u.values*units.meter/units.second, 
                                                        ds[a].v.values*units.meter/units.second)).m)
        ds[a].wspd.attrs['long_name'] = 'Wind Speed'
ds[0].data_vars

Year (yyyy): 2024
Month (mm): 04
Day (dd): 25
Enter pressure level (surface, 1000 mb, 925 mb, 850 mb, etc.): 850 mb
Choose the number of forecast hours (up to 48): 48


✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F00 ┊ GRIB2 @ local ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F05 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F07 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F08 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F09 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2024-Apr-25 00:00 UTC F10 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrr

Data variables:
    unknown  (y, x) float32 8MB ...
    t        (y, x) float32 8MB ...
    u        (y, x) float32 8MB -1.034 -0.9717 -0.9717 ... 4.278 4.278 4.278
    v        (y, x) float32 8MB -2.627 -2.564 -2.502 ... 0.8732 0.9357 0.9982
    q        (y, x) float32 8MB ...
    w        (y, x) float32 8MB ...
    gh       (y, x) float32 8MB ...
    r        (y, x) float32 8MB ...
    dpt      (y, x) float32 8MB ...
    absv     (y, x) float32 8MB ...
    clwmr    (y, x) float32 8MB ...
    rwmr     (y, x) float32 8MB ...
    snmr     (y, x) float32 8MB ...
    grle     (y, x) float32 8MB ...
    wspd     (y, x) float32 8MB 2.823 2.742 2.684 2.603 ... 4.366 4.379 4.393

### Choose Variable
Choose a variable from the list above.  Once you enter it, you will get the full name of the variable.  Geopotential heights (gh) should not be chosen here, as they will already be plotted as contours for upper air maps.

In [3]:
var = input('Choose your variable (for cfill):')   
ds[0][f'{var}'].long_name

Choose your variable (for cfill): dpt


'Dew point temperature'

### Plot the Data
Here we will plot the data, with the three sites denoted by the black stars.

In [4]:
#These if statements determine the colormap, colormap scale, and contour interval
#that will be used depending on what variable and at which level is being plotted.
if level == str('200 mb') or level == str('250 mb') or level == str('300 mb'):
    comp_scale = np.arange(-70, 71, 10)
    wind_scale = np.arange(30, 81, 10)
    i = 120
elif level == str('500 mb'):
    comp_scale = np.arange(-40, 41, 5)
    wind_scale = np.arange(0, 51, 10)
    i = 60
elif level == str('700 mb') or level == str('850 mb'):
    comp_scale = np.arange(-25, 26, 5)
    wind_scale = np.arange(0, 31, 5)
    i = 30
elif level == str('925 mb') or level == str('1000 mb'):
    comp_scale = np.arange(-20, 20.1, 2)
    wind_scale = np.arange(0, 21, 2)
    i = 20
if var == str('t'):
    cmap = plt.cm.coolwarm
    scale = np.arange(210, 321, 5)
elif var == str('q'):
    cmap = plt.cm.Greens
    scale = np.arange(0.0001, 0.0005, 0.000001)
elif var == str('dpt'):
    cmap = plt.cm.Greens
    scale = np.arange(235, 311, 5)
elif var == str('r'):
    cmap = plt.cm.Greens
    scale = np.arange(0, 101, 5)
elif var == str('clwmr') or var == str('snmr') or var == str('rwmr'):
    cmap = plt.cm.BuGn
    scale = np.arange(0, 5e-3, 1e-6)
elif var == str('absv'):
    cmap = plt.cm.PuOr_r
    scale = np.arange(-0.001, 0.00101, 0.0001)
elif var == str('u') or var == str('v'):
    cmap = plt.cm.BuPu
    scale = comp_scale
elif var == str('wspd'):
    cmap = plt.cm.BuPu
    scale = wind_scale
elif var == str('w'):
    cmap = plt.cm.BrBG
    scale = np.arange(-4.5, 4.51, 0.05)
elif var == str('grle'):
    cmap = plt.cm.Blues
    scale = np.arange(0, 5e-3, 1e-6)
elif var == str('cape'):
    scale = np.arange(0, 7001, 500)
    cmap = plt.cm.Reds
elif var == str('cin'):
    scale = np.arange(-300, 301, 25)
    cmap = plt.cmap.gist_heat_r
elif var == str('vis'):
    scale = np.arange(0, 100001, 5000)
    cmap = plt.cm.Greys_r
elif var == str('frzr'):
    scale = np.arange(0, 0.51, 0.01)
    cmap = plt.cm.RdPu
elif var == str('sde'):
    scale = np.arange(0, 1.01, 0.01)
    cmap = plt.cm.Blues
elif var == str('blh'):
    scale = np.arange(0, 5001, 100)
    cmap = plt.cm.Oranges_r
elif var == str('hail'):
    scale = np.arange(0, 0.251, 0.01)
    cmap = plt.cm.Greys
elif var_sfc == str('gust'):
    scale = np.arange(0, 36, 5)
    cmap = plt.cm.BuPu
elif var_sfc == str('dswrf') or var_sfc == str('uswrf') or var_sfc == str('dlwrf') or var_sfc == str('ulwrf'):
    scale = np.arange(0, 601, 25)
    cmap = plt.cm.viridis
elif var_sfc == str('gflux'):
    scale = np.arange(-750, 751, 50)
    cmap = plt.cm.viridis

In [5]:
#Coordinates:
#RI: 41.4456, -71.4357
#CC: 42.03, -70.049
#Sodar: 41.2453, -70.105

#Plot the figures
for a in range(0, int(fxx) + 1):
    fig = plt.figure(1, figsize=(15, 15))
    ax = plt.subplot(111, projection=ccrs.PlateCarree())
    #ax.set_extent([-69.5, -72, 42.5, 41]), ccrs.PlateCarree() 
    ax.set_extent([-125, -65, 25, 47]), ccrs.PlateCarree()
    cf = ax.contourf(ds[0].longitude, ds[0].latitude, ds[a][f'{var}'], scale, cmap=cmap)
    if product == str('prs'):
        cs = ax.contour(ds[0].longitude, ds[0].latitude, ds[a].gh, np.arange(0, 15000, i), 
                        colors='black', transform=ccrs.PlateCarree())
        plt.clabel(cs)
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))
    ax.scatter(-71.4357, 41.4456, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    ax.scatter(-70.049, 42.03, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    ax.scatter(-70.105, 41.2453, 400, marker='*', color='black', transform=ccrs.PlateCarree())
    plt.title(f'Time: {date} FH {a}   HRRR {ds[0][f'{var}'].long_name} at {level}')
    #plt.show()
    plt.savefig(f'../../../Downloads/{yr}{m}{d}_{var}_{level}_FH_{fh[a]}_HRRR.png', bbox_inches='tight', dpi=150)
    plt.close()

### Making a Gif
We will use the saved images to create a looping gif for the forecast period.

In [6]:
# List of image filenames
filenames = sorted(glob.glob(f"../../../Downloads/{yr}{m}{d}_{var}_{level}*.png"))
# Create GIF
images = [imageio.imread(filename) for filename in filenames]
imageio.mimsave(f"../../../Downloads/{yr}{m}{d}_loop_HRRR_{var}_{level}.gif", images, loop=1000)

C:\Users\nickm\AppData\Local\Temp\ipykernel_22376\3247646894.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(filename) for filename in filenames]
